In [1]:
import pandas as pd
import numpy as np

In [3]:
from harmonic_inference.data.corpus_reading import aggregate_annotation_dfs
from pathlib import Path

ANNOTATIONS_PATH = Path('../corpora/annotations')
OUT_DIR = Path('corpus_data')

aggregate_annotation_dfs(ANNOTATIONS_PATH, OUT_DIR)

100%|██████████| 749/749 [00:26<00:00, 28.32it/s]


In [2]:
from harmonic_inference.data.corpus_reading import read_dump
from pathlib import Path

files_df = read_dump(Path('corpus_data', 'files.tsv'), index_col=0)
measures_df = read_dump(Path('corpus_data', 'measures.tsv'))
chords_df = read_dump(Path('corpus_data', 'chords.tsv'))
notes_df = read_dump(Path('corpus_data', 'notes.tsv'))

In [3]:
from harmonic_inference.utils import corpus_utils as cu
import importlib
importlib.reload(cu)

# Remove measure repeats
if isinstance(measures_df.iloc[0].next, tuple):
    measures_df = cu.remove_repeats(measures_df)

# Add offsets
if not all([column in notes_df.columns for column in ['offset_beat', 'offset_mc']]):
    offset_mc, offset_beat = cu.get_offsets(notes_df, measures_df)
    notes_df = notes_df.assign(offset_mc=offset_mc, offset_beat=offset_beat)



In [2]:
from harmonic_inference.data.corpus_reading import read_dump
from harmonic_inference.utils import corpus_utils as cu

chords_tsv = 'data/chord_list.tsv'
notes_tsv = 'data/note_list.tsv'
measures_tsv = 'data/measure_list.tsv'
files_tsv = 'data/file_list.tsv'

chords_df = read_dump(chords_tsv)
notes_df = read_dump(notes_tsv, index_col=[0,1,2])
measures_df = read_dump(measures_tsv)
files_df = read_dump(files_tsv, index_col=0)

# Bugfixes
measures_df.loc[(685, 487), 'next'][0] = 488

# Remove measure repeats
if isinstance(measures_df.iloc[0].next, list):
    measures_df = cu.remove_repeats(measures_df)

# Add offsets
if not all([column in notes_df.columns for column in ['offset_beat', 'offset_mc']]):
    offset_mc, offset_beat = cu.get_offsets(notes_df, measures_df)
    notes_df = notes_df.assign(offset_mc=offset_mc, offset_beat=offset_beat)

# Merge ties
notes_df = cu.merge_ties(notes_df, measures=measures_df)

In [3]:
files_df.loc[0]

extension                                     .mscx
filename                     BWV806_01_Prélude.mscx
md5                e81d2b409a3c696e733ac86b86f522ee
name                              BWV806_01_Prélude
rel_path     annotations/Bach_Suites/English Suites
subcorpus                            English Suites
Name: 0, dtype: object

In [4]:
chords_df.loc[0]

,mc,mn,onset,chords,chord,key,pedal,numeral,form,figbass,...,relativeroot,phraseend,globalkey,bass_step,mc_next,onset_next,next_chord_id,chord_length,globalminor,localminor
chord_id,,,,,,,,,,,,,,,,,,,,,
0,0,1,0,.A.I,I,I,<NA>,I,<NA>,<NA>,...,<NA>,<NA>,A,1,0,3/8,1,3/8,False,False
1,0,1,3/8,V7,V7,I,<NA>,V,<NA>,7,...,<NA>,<NA>,A,5,0,3/4,2,3/8,False,False
2,0,1,3/4,I,I,I,<NA>,I,<NA>,<NA>,...,<NA>,<NA>,A,1,0,9/8,3,3/8,False,False
3,0,1,9/8,V7,V7,I,<NA>,V,<NA>,7,...,<NA>,<NA>,A,5,1,0,4,3/8,False,False
4,1,2,0,I,I,I,<NA>,I,<NA>,<NA>,...,<NA>,<NA>,A,1,1,3/8,5,3/8,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,36,37,0,IV6,IV6,I,<NA>,IV,<NA>,6,...,<NA>,<NA>,A,6,36,3/8,229,3/8,False,False
229,36,37,3/8,IV64,IV64,I,<NA>,IV,<NA>,64,...,<NA>,<NA>,A,1,36,1/2,230,1/8,False,False
230,36,37,1/2,I,I,I,<NA>,I,<NA>,<NA>,...,<NA>,<NA>,A,1,36,5/8,231,1/8,False,False


In [5]:
measures_df.loc[0]

,section,keysig,timesig,duration,act_dur,offset,voices,repeats,volta,barline,numbering_offset,dont_count,marker,mn,next
mc,,,,,,,,,,,,,,,
0,0,3,12/8,3/2,3/2,0,4,firstMeasure,<NA>,<NA>,<NA>,<NA>,<NA>,1,1
1,0,3,12/8,3/2,3/2,0,5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,2
2,0,3,12/8,3/2,3/2,0,6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3,3
3,0,3,12/8,3/2,3/2,0,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4,4
4,0,3,12/8,3/2,3/2,0,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,5,5
5,0,3,12/8,3/2,3/2,0,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,6,6
6,0,3,12/8,3/2,3/2,0,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7,7
7,0,3,12/8,3/2,3/2,0,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,8,8
8,0,3,12/8,3/2,3/2,0,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,9,9


In [6]:
notes_df.loc[0]

mc  mn onset duration gracenote nominal_duration scalar  tied  \
section ix                                                                   
0       0     0   1     0     1/16      <NA>             1/16      1  <NA>   
        1     0   1  1/16     1/16      <NA>             1/16      1  <NA>   
        2     0   1   1/8     1/16      <NA>             1/16      1  <NA>   
        3     0   1  3/16     1/16      <NA>             1/16      1  <NA>   
        4     0   1   1/4     1/16      <NA>             1/16      1  <NA>   
...          ..  ..   ...      ...       ...              ...    ...   ...   
        860  36  37   5/8      1/8      <NA>              1/8      1  <NA>   
        861  36  37   5/8      1/8      <NA>              1/8      1  <NA>   
        862  36  37   3/4      3/4      <NA>              1/2    3/2  <NA>   
        864  36  37   3/4      3/4      <NA>              1/2    3/2  <NA>   
        865  36  37   3/4      3/4      <NA>              1/2    3/2  <NA>   

             tpc  midi  staff  voice  volta  octaves note_names  keysig  \
section ix                                                                
0       0      3    45      2      2   <NA>        2          A       3   
        1      7    49      2      2   <NA>        3         C#       3   
        2      4    52      2      2   <NA>        3          E       3   
        3      3    57      2      1   <NA>        3          A       3   
        4      8    56      2      1   <NA>        3         G#       3   
...          ...   ...    ...    ...    ...      ...        ...     ...   
        860    5    71      1      2   <NA>        4          B       3   
        861    2    74      1      3   <NA>        5          D       3   
        862    3    45      2      2   <NA>        2          A       3   
        864    7    73      1      2   <NA>        5         C#       3   
        865    4    76      1      3   <NA>        5          E       3   

            timesig  beats offset_mc offset_beat  
section ix                                        
0       0      12/8      1         0        1/16  
        1      12/8  1.1/6         0         1/8  
        2      12/8  1.1/3         0        3/16  
        3      12/8  1.1/2         0         1/4  
        4      12/8  1.2/3         0        5/16  
...             ...    ...       ...         ...  
        860    12/8  2.2/3        36         3/4  
        861    12/8  2.2/3        36         3/4  
        862    12/8      3        36         3/2  
        864    12/8      3        36         3/2  
        865    12/8      3        36         3/2  

[774 rows x 20 columns]

In [11]:
import harmonic_inference.data.piece as piece
import harmonic_inference.utils.harmonic_utils as hu

import importlib
importlib.reload(piece)
importlib.reload(hu)

score_piece = piece.ScorePiece(notes_df.loc[0], chords_df.loc[0], measures_df.loc[0])
for key in score_piece.get_keys():
    print(key)

A KeyMode.MAJOR
D KeyMode.MAJOR
A KeyMode.MAJOR
D KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
D KeyMode.MAJOR
A KeyMode.MAJOR
F# KeyMode.MINOR
A KeyMode.MAJOR
F# KeyMode.MINOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
F# KeyMode.MINOR
A KeyMode.MAJOR
F# KeyMode.MINOR
A KeyMode.MAJOR
F# KeyMode.MINOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
F# KeyMode.MINOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
F# KeyMode.MINOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
A KeyMode.MAJOR
E KeyMode.MAJOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
E KeyMode.MAJOR
A

In [9]:
chords_df.loc[94].loc[chords_df.loc[94].mc == 71]

,mc,mn,onset,chords,chord,key,pedal,numeral,form,figbass,...,relativeroot,phraseend,globalkey,bass_step,mc_next,onset_next,next_chord_id,chord_length,globalminor,localminor
chord_id,,,,,,,,,,,,,,,,,,,,,
14695,71,72,0,#viio6/III,#viio6/III,I,<NA>,#vii,o,6,...,III,<NA>,E,##4,71,1/4,14696,1/4,False,False
14696,71,72,1/4,III,III,I,<NA>,III,<NA>,<NA>,...,<NA>,<NA>,E,3,71,7/16,14697,3/16,False,False
14697,71,72,7/16,V7/V,V7/V,I,<NA>,V,<NA>,7,...,V,<NA>,E,2,72,1/16,14698,1/8,False,False


In [16]:
files_df.loc[102]

extension                               .mscx
filename                            08-1.mscx
md5          b69eef6286168c9d334cd29ce4bae9dd
name                                     08-1
rel_path        annotations/Beethoven-Sonatas
subcorpus                   Beethoven-Sonatas
Name: 102, dtype: object

In [ ]:
import harmonic_inference_data as hid
import importlib
importlib.reload(hid)

datasets = {'global': {},
            'local': {},
            'none': {}}

datasets['global']['train'], datasets['global']['valid'], datasets['global']['test'] = hid.get_train_valid_test_splits(
    chords_df=chords_df, notes_df=notes_df, measures_df=measures_df, files_df=files_df,
    seed=0, h5_directory='data', h5_prefix='globalkey_811split', make_dfs=True, transpose_global=True
)

datasets['local']['train'], datasets['local']['valid'], datasets['local']['test'] = hid.get_train_valid_test_splits(
    chords_df=chords_df, notes_df=notes_df, measures_df=measures_df, files_df=files_df,
    seed=0, h5_directory='data', h5_prefix='localkey_811split', make_dfs=True, transpose_local=True
)

datasets['none']['train'], datasets['none']['valid'], datasets['none']['test'] = hid.get_train_valid_test_splits(
    chords_df=chords_df, notes_df=notes_df, measures_df=measures_df, files_df=files_df,
    seed=0, h5_directory='data', h5_prefix='811split', make_dfs=True
)

In [ ]:
import harmonic_inference_models as him
import harmonic_utils
import importlib
importlib.reload(him)
from ablation import get_masks_and_names
import torch
import os

masks, mask_names = get_masks_and_names()
dir_name = 'results'

data = 'local'
index = -1
prefix = (data + '_') if data != 'none' else ''

mask = torch.tensor(masks[index])
mask_name = mask_names[index]
checkpoint_dir = os.path.join(dir_name, prefix + mask_name)
log_file_name = os.path.join(dir_name, prefix + mask_name + '.log')

model = him.MusicScoreModel(len(datasets[data]['test'][0]['notes'][0]), len(harmonic_utils.CHORD_TYPES) * 12,
                            dropout=0.2, input_mask=mask)

print(mask_name)
print(checkpoint_dir)

In [ ]:
import os
import model_trainer
import importlib
importlib.reload(model_trainer)

from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

optimizer = Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001)
criterion = CrossEntropyLoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
schedule_var = 'valid_loss'

trainer = model_trainer.ModelTrainer(model, train_dataset=datasets[data]['train'], valid_dataset=datasets[data]['valid'],
                                     test_dataset=datasets[data]['test'], seed=0, num_epochs=100, early_stopping=20,
                                     optimizer=optimizer, scheduler=scheduler, schedule_var=schedule_var,
                                     criterion=criterion,
                                     log_every=1, log_file_name=log_file_name,
                                     save_every=10, save_dir=checkpoint_dir, save_prefix='checkpoint',
                                     resume=os.path.join(checkpoint_dir, 'best.pth.tar'))

In [ ]:
trainer.train()

In [ ]:
trainer = model_trainer.ModelTrainer(model, train_dataset=datasets[data]['train'], valid_dataset=datasets[data]['valid'],
                                     test_dataset=datasets[data]['test'], seed=0, num_epochs=100, early_stopping=20,
                                     optimizer=optimizer, scheduler=scheduler, schedule_var=schedule_var,
                                     criterion=criterion,
                                     log_every=1, log_file_name=log_file_name,
                                     save_every=10, save_dir=checkpoint_dir, save_prefix='checkpoint',
                                     resume=os.path.join(checkpoint_dir, 'best.pth.tar'))

loss, acc, outputs, labels = trainer.evaluate(valid=False)
print(loss, acc)

In [ ]:
import eval_utils as eu
import harmonic_utils as hu
import matplotlib.pyplot as plt

label_strings = hu.get_one_hot_labels()
conf_mat = eu.get_conf_mat(labels, outputs)

plt.figure(figsize=(30,30))
plt.imshow(conf_mat, interpolation='none')
plt.colorbar()
plt.xticks(ticks=list(range(len(label_strings))), labels=label_strings, rotation=90, fontsize=10)
plt.yticks(ticks=list(range(len(label_strings))), labels=label_strings, fontsize=10)
plt.show()

In [ ]:
import eval_utils as eu

correct, incorrect = eu.get_correct_and_incorrect_indexes(labels, outputs)
print('Correct: ' + str(len(correct)))
print('Incorrect: ' + str(len(incorrect)))

In [ ]:
import eval_utils as eu
    
eu.print_result(incorrect[0], labels, outputs, limit=10, prob=False)

In [ ]:
import eval_utils as eu

chord, onset_notes, all_notes = eu.get_input_df_rows(incorrect[0], datasets[data]['test'])

print(chord)
print("USED NOTES:")
print(onset_notes)
print()
print("ALL NOTES:")
print(all_notes)

In [ ]:
import matplotlib.pyplot as plt
import eval_utils as eu

correct_ranks, indexes_by_rank = eu.get_correct_ranks(labels, outputs)
    
plt.figure(figsize=(30,30))
plt.bar(range(len(outputs[0])), [len(indexes) for indexes in indexes_by_rank])

In [ ]:
import eval_utils as eu
import importlib
importlib.reload(eu)

eval_df = eu.get_eval_df(labels, outputs, datasets[data]['test'])
eval_df

In [ ]:
import ablation
import importlib
importlib.reload(ablation)

dfs = ablation.load_all_ablated_dfs(directory='results', prefix=prefix[:-1] if len(prefix) > 0 else None)
_, mask_names = ablation.get_masks_and_names()

In [ ]:
import pandas as pd
import os

logs = []
for mask_name in mask_names:
    logs.append(pd.read_csv(os.path.join(os.path.join('results', prefix + mask_name + '.log'))))

In [ ]:
for df, log, mask_name in zip(dfs, logs, mask_names):
    print(f"{mask_name} Acc: {100 * df.correct.sum() / len(df)}")
    print(log.iloc[-1])

In [ ]:
import eval_utils as eu

global_df = eu.load_eval_df('results/global_no_ablation.csv')
local_df = eu.load_eval_df('results/local_no_ablation.csv')
none_df = eu.load_eval_df('results/no_ablation.csv')

In [ ]:
global_df

In [ ]:
global_counts = global_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)
local_counts = local_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)
none_counts = none_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.scatter(global_counts['count'], global_counts['mean'], color='red', label='Global key')
plt.scatter(local_counts['count'], local_counts['mean'], color='blue', label='Local key')
plt.scatter(none_counts['count'], none_counts['mean'], color='yellow', label='No transposition')
plt.title('Global key transposed')
plt.xlabel('Count')
plt.ylabel('Accuracy')
plt.legend()
plt.show()